In [1]:
import subprocess
import sys
import random
import torch
import numpy as np

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

def check_and_install(package, import_name=None):
    try:
        if import_name is None:
            __import__(package)
        else:
            __import__(import_name)
    except ImportError:
        install(package)

# Libraries for DonaDev
libraries = {
    'torch': 'torch',
    'torchvision': 'torchvision',
    'Pillow': 'PIL',
    'requests': 'requests',
    'gitpython': 'git',
    'matplotlib': 'matplotlib',
}

for package, import_name in libraries.items():
    check_and_install(package, import_name)

print("All necessary libraries are installed.")

# Your DonaDev code
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import MNIST, CIFAR10
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import io
import requests
import time
import os
from git import Repo
import subprocess
import json

class AIDev:
    def __init__(self, learning_rate=0.001, batch_size=64, num_epochs=2, hidden_size=512, image_size=(28, 28), input_channels=1, random_seed=42):
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.hidden_size = hidden_size
        self.image_size = image_size
        self.input_channels = input_channels
        self.random_seed = random_seed
        self.model = None
        self.train_loader = None
        self.criterion = None
        self.optimizer = None
        self.set_random_seed(random_seed)

    def set_random_seed(self, seed):
        self.random_seed = seed
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

    class SimpleNet(nn.Module):
        def __init__(self, hidden_size, input_size):
            super().__init__()
            self.flatten = nn.Flatten()
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(input_size, hidden_size),
                nn.ReLU(),
                nn.Linear(hidden_size, 10)
            )

        def forward(self, x):
            x = self.flatten(x)
            return self.linear_relu_stack(x)

    def modelinit(self, dataset_choice, data_dir=None):
        if dataset_choice == "mnist":
            self.image_size = (28, 28)
            self.input_channels = 1
            transform = transforms.Compose([
                transforms.Resize(self.image_size),
                transforms.ToTensor(),
                transforms.Normalize((0.5,), (0.5,))
            ])
            train_dataset = MNIST(root='./data', train=True, transform=transform, download=True)
        elif dataset_choice == "cifar10":
            self.image_size = (32, 32)
            self.input_channels = 3
            transform = transforms.Compose([
                transforms.Resize(self.image_size),
                transforms.ToTensor(),
                transforms.Normalize((0.5,), (0.5,))
            ])
            train_dataset = CIFAR10(root='./data', train=True, transform=transform, download=True)
        else:
            self.input_channels = 3  # Assuming custom data is RGB by default
            transform = transforms.Compose([
                transforms.Resize(self.image_size),
                transforms.ToTensor(),
                transforms.Normalize((0.5,), (0.5,))
            ])
            train_dataset = ImageFolder(root=data_dir, transform=transform)

        self.train_loader = DataLoader(dataset=train_dataset, batch_size=self.batch_size, shuffle=True)
        input_size = self.image_size[0] * self.image_size[1] * self.input_channels
        self.model = self.SimpleNet(self.hidden_size, input_size)
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        return self.model

    def train(self):
        self.model.train()
        epoch_losses = []
        for epoch in range(self.num_epochs):
            epoch_loss = 0
            for _, (data, targets) in enumerate(self.train_loader):
                outputs = self.model(data)
                loss = self.criterion(outputs, targets)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                epoch_loss += loss.item()
            epoch_loss /= len(self.train_loader)
            epoch_losses.append(epoch_loss)
            print(f'Epoch {epoch + 1}, Loss: {epoch_loss:.4f}')
        return epoch_losses

    def preprocess_image(self, imgbytes):
        image_stream = io.BytesIO(imgbytes)
        image = Image.open(image_stream).convert('RGB' if self.input_channels == 3 else 'L')
        transform = transforms.Compose([
            transforms.Resize(self.image_size),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])
        image = transform(image).unsqueeze(0)
        return image

    def test(self, image_bytes):
        image_tensor = self.preprocess_image(image_bytes)
        self.model.eval()
        res = self.model(image_tensor)
        predicted_class = res.argmax(dim=1).item()
        print(f'Predicted class: {predicted_class}')
        return predicted_class

    def datatype(self):
        return "images"

    def plot_losses(self, epoch_losses):
        plt.figure(figsize=(10, 5))
        plt.plot(epoch_losses, marker='o', linestyle='-', color='b')
        plt.title('Training Loss Over Epochs')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.grid(True)
        plt.savefig('training_loss_plot.png')
        plt.close()

    def save_log(self, hyperparameters, final_loss, random_seed):
        log = {
            "hyperparameters": hyperparameters,
            "final_loss": final_loss,
            "random_seed": random_seed
        }
        with open('training_logs.json', 'a') as log_file:
            log_file.write(json.dumps(log) + '\n')

    def show_logs(self):
        if not os.path.exists('training_logs.json'):
            return []

        with open('training_logs.json', 'r') as log_file:
            logs = log_file.readlines()
            logs = [json.loads(log) for log in logs]
            return logs


class DonaDev:
    def __init__(self, token, chat_id, ai_dev, data_dir):
        self.TOKEN = token
        self.chat_id = chat_id
        self.ai_dev = ai_dev
        self.data_dir = data_dir
        self.last_update_id = None

    def send_telegram_message(self, message):
        url = f"https://api.telegram.org/bot{self.TOKEN}/sendMessage?chat_id={self.chat_id}&text={message}"
        response = requests.get(url)
        print(f"Sent message: {message}, Response: {response.json()}")

    def send_telegram_image(self, image_path):
        url = f"https://api.telegram.org/bot{self.TOKEN}/sendPhoto"
        with open(image_path, 'rb') as image_file:
            files = {'photo': image_file}
            data = {'chat_id': self.chat_id}
            response = requests.post(url, files=files, data=data)
        print(f"Sent image: {image_path}, Response: {response.json()}")

    def get_updates(self, offset=None):
        url = f"https://api.telegram.org/bot{self.TOKEN}/getUpdates"
        params = {'offset': offset, 'timeout': 30}
        response = requests.get(url, params=params).json()
        return response.get('result', [])

    def clear_previous_messages(self):
        updates = self.get_updates()
        if updates:
            self.last_update_id = updates[-1]['update_id'] + 1
        print(f"Cleared previous messages. Last update ID: {self.last_update_id}")

    def get_cuda_options(self):
        p = subprocess.run(['nvidia-smi', '-q'], capture_output=True, text=True)
        pret = p.returncode
        if pret != 0:
            print("Device doesn't have GPU or CUDA drivers error")
            self.send_telegram_message("Device doesn't have GPU or CUDA drivers error")
            return
        pop = p.stdout
        for popline in pop.splitlines():
            self.send_telegram_message(popline)

    def request_user_input(self, prompt):
        self.send_telegram_message(prompt)
        start_time = time.time()
        timeout = 300
        while time.time() - start_time < timeout:
            updates = self.get_updates(self.last_update_id)
            for update in updates:
                self.last_update_id = update['update_id'] + 1
                if 'message' in update and 'text' in update['message']:
                    return update['message']['text'].lower()
                if 'message' in update and 'photo' in update['message']:
                    image_id = update['message']['photo'][0]['file_id']
                    file_info = requests.get(f"https://api.telegram.org/bot{self.TOKEN}/getFile?file_id={image_id}").json()
                    file_path = file_info['result']['file_path']
                    image_bytes = requests.get(f"https://api.telegram.org/file/bot{self.TOKEN}/{file_path}").content
                    return image_bytes
            time.sleep(1)
        self.send_telegram_message("No input received within 5 minutes. Using default value.")
        return None

    def get_hyperparameters(self, dataset_choice):
        if dataset_choice == 'mnist':
            default_lr = 0.001
            default_bs = 64
            default_epochs = 2
            default_hidden_size = 512
            default_image_size = (28, 28)
            default_input_channels = 1
        elif dataset_choice == 'cifar10':
            default_lr = 0.001
            default_bs = 64
            default_epochs = 2
            default_hidden_size = 512
            default_image_size = (32, 32)
            default_input_channels = 3
        else:
            default_lr = 0.001
            default_bs = 64
            default_epochs = 2
            default_hidden_size = 512
            default_image_size = (28, 28)
            default_input_channels = 3  # Assuming custom data follows CIFAR-10 size by default

        choice = self.request_user_input("Do you want to input hyperparameters? (yes/no)")
        if choice == 'yes':
            learning_rate = float(self.request_user_input(f"Enter learning rate (e.g., {default_lr}):") or default_lr)
            batch_size = int(self.request_user_input(f"Enter batch size (e.g., {default_bs}):") or default_bs)
            num_epochs = int(self.request_user_input(f"Enter number of epochs (e.g., {default_epochs}):") or default_epochs)
            hidden_size = int(self.request_user_input(f"Enter hidden layer size (e.g., {default_hidden_size}):") or default_hidden_size)
            image_size = tuple(map(int, self.request_user_input(f"Enter image size (e.g., {default_image_size[0]}, {default_image_size[1]}):").split(',')) or default_image_size)
            input_channels = int(self.request_user_input(f"Enter input channels (e.g., {default_input_channels}):") or default_input_channels)
        else:
            learning_rate = default_lr
            batch_size = default_bs
            num_epochs = default_epochs
            hidden_size = default_hidden_size
            image_size = default_image_size
            input_channels = default_input_channels

        self.ai_dev.learning_rate = learning_rate
        self.ai_dev.batch_size = batch_size
        self.ai_dev.num_epochs = num_epochs
        self.ai_dev.hidden_size = hidden_size
        self.ai_dev.image_size = image_size
        self.ai_dev.input_channels = input_channels
        self.send_telegram_message(f"Using hyperparameters: learning_rate={learning_rate}, batch_size={batch_size}, num_epochs={num_epochs}, hidden_size={hidden_size}, image_size={image_size}, input_channels={input_channels}")

    def check_file_sizes(self):
        max_file_size = 100 * 1024 * 1024
        acceptable_files = []
        for root, dirs, files in os.walk('.'):
            for file in files:
                file_path = os.path.join(root, file)
                if os.path.getsize(file_path) <= max_file_size:
                    acceptable_files.append(file_path)
        return acceptable_files

    def push_to_github(self, repo_url, is_private):
        try:
            if os.path.exists('.git'):
                repo = Repo('.')
            else:
                repo = Repo.init('.')
                repo.create_remote('origin', url=repo_url)
            repo.git.add(A=True)
            repo.index.commit("Update from Telegram bot")
            origin = repo.remote('origin')
            origin.push()
            self.send_telegram_message("Successfully pushed to GitHub repository.")
        except Exception as e:
            self.send_telegram_message(f"Error pushing to GitHub: {str(e)}")

    def collect_data(self):
        while True:
            label_choice = self.request_user_input("Does the label folder already exist? (yes/no)")
            if label_choice == "yes":
                existing_labels = os.listdir(self.data_dir)
                self.send_telegram_message(f"Existing labels: {', '.join(existing_labels)}")
                label = self.request_user_input("Enter the label folder name:")
                if label not in existing_labels:
                    self.send_telegram_message("Error: Label folder does not exist.")
                    continue
            else:
                label = self.request_user_input("Enter the new label folder name:")
                os.makedirs(os.path.join(self.data_dir, label), exist_ok=True)

            while True:
                image_bytes = self.request_user_input(f"Upload an image for the label '{label}':")
                if isinstance(image_bytes, bytes):
                    image_path = os.path.join(self.data_dir, label, f"{int(time.time())}.png")
                    with open(image_path, 'wb') as f:
                        f.write(image_bytes)
                    self.send_telegram_message(f"Image saved to {image_path}")
                else:
                    break

            next_action = self.request_user_input("Type 'stop' to finish collecting data, or 'continue' to add more images:")
            if next_action == "stop":
                break

    def show_random_seed_logs(self):
        logs = self.ai_dev.show_logs()
        if not logs:
            self.send_telegram_message("No logs found.")
            return

        message = "Random Seed Logs:\n"
        for log in logs:
            message += (f"Random Seed: {log['random_seed']}, Loss: {log['final_loss']}, "
                        f"Hyperparameters: {log['hyperparameters']}\n")
        self.send_telegram_message(message)

    def main_loop(self):
        self.clear_previous_messages()
        while True:
            dataset_choice = self.request_user_input("Do you want to use package-based data or custom data? (package/custom)")
            if dataset_choice == "stop":
                self.send_telegram_message("Process stopped by user command.")
                break
            while dataset_choice not in ["package", "custom"]:
                self.send_telegram_message("Invalid choice. Please enter 'package' or 'custom'.")
                dataset_choice = self.request_user_input("Do you want to use package-based data or custom data? (package/custom)")
                if dataset_choice == "stop":
                    self.send_telegram_message("Process stopped by user command.")
                    break

            if dataset_choice == "package":
                dataset_type = self.request_user_input("Choose a dataset (mnist/cifar10):")
                while dataset_type not in ["mnist", "cifar10"]:
                    self.send_telegram_message("Invalid dataset choice. Please choose 'mnist' or 'cifar10'.")
                    dataset_type = self.request_user_input("Choose a dataset (mnist/cifar10):")
                    if dataset_type == "stop":
                        self.send_telegram_message("Process stopped by user command.")
                        break
                if dataset_type == "stop":
                    break
                next_action = "train"  # Proceed to training after valid dataset choice
            else:
                dataset_type = "custom"
                self.collect_data()
                next_action = self.request_user_input("Data collection complete. Do you want to train or stop? (train/stop)")
                while next_action not in ["train", "stop"]:
                    self.send_telegram_message("Invalid choice. Please enter 'train' or 'stop'.")
                    next_action = self.request_user_input("Data collection complete. Do you want to train or stop? (train/stop)")
                if next_action == "stop":
                    self.send_telegram_message("Process stopped by user command.")
                    break

            if next_action == "train":
                self.show_random_seed_logs()
                seed_choice = self.request_user_input("Do you want to set a custom random seed? (yes/no)")
                if seed_choice == "yes":
                    random_seed = int(self.request_user_input("Enter the random seed value:"))
                    self.ai_dev.set_random_seed(random_seed)
                self.get_hyperparameters(dataset_type)
                self.ai_dev.modelinit(dataset_type, self.data_dir if dataset_type == "custom" else None)
                self.send_telegram_message("Model training has started.")
                epoch_losses = self.ai_dev.train()
                final_loss = epoch_losses[-1]
                hyperparameters = {
                    "learning_rate": self.ai_dev.learning_rate,
                    "batch_size": self.ai_dev.batch_size,
                    "num_epochs": self.ai_dev.num_epochs,
                    "hidden_size": self.ai_dev.hidden_size,
                    "image_size": self.ai_dev.image_size,
                    "input_channels": self.ai_dev.input_channels
                }
                self.ai_dev.save_log(hyperparameters, final_loss, self.ai_dev.random_seed)
                self.send_telegram_message(f"Training complete. Final loss: {final_loss:.4f}")
                self.ai_dev.plot_losses(epoch_losses)
                self.send_telegram_image('training_loss_plot.png')

                use_github = self.request_user_input("Do you want to use GitHub? (yes/no)")
                while use_github not in ["yes", "no"]:
                    self.send_telegram_message("Invalid choice. Please enter 'yes' or 'no'.")
                    use_github = self.request_user_input("Do you want to use GitHub? (yes/no)")
                if use_github == 'yes':
                    acceptable_files = self.check_file_sizes()
                    self.send_telegram_message(f"Files acceptable for GitHub push: {', '.join(acceptable_files)}")
                    push_decision = self.request_user_input("Do you want to push these files to GitHub? (yes/no)")
                    while push_decision not in ["yes", "no"]:
                        self.send_telegram_message("Invalid choice. Please enter 'yes' or 'no'.")
                        push_decision = self.request_user_input("Do you want to push these files to GitHub? (yes/no)")
                    if push_decision == 'yes':
                        repo_url = self.request_user_input("Enter your GitHub repository URL:")
                        is_private = self.request_user_input("Is this a private repository? (yes/no)") == 'yes'
                        self.push_to_github(repo_url, is_private)

                command = self.request_user_input("Enter 'rerun' to train again with new parameters, or 'stop' to end the program, 'cuda' to get CUDA status and stop, or an image to test model:")
                while command not in ["rerun", "stop", "cuda"]:
                    self.send_telegram_message("Invalid choice. Please enter 'rerun', 'stop', or 'cuda'.")
                    command = self.request_user_input("Enter 'rerun' to train again with new parameters, or 'stop' to end the program, 'cuda' to get CUDA status and stop, or an image to test model:")
                if command == "stop":
                    self.send_telegram_message("Training stopped by user command.")
                    break
                elif command == "rerun":
                    self.send_telegram_message("Rerunning the training with new parameters.")
                    continue
                elif command == "cuda":
                    self.get_cuda_options()
                    break

ai_dev = AIDev()
dona_dev = DonaDev(token='7166875544:AAGBi7azOdXowUbrZKXHpm7p152pG-mDxkA', chat_id='5255840420', ai_dev=ai_dev, data_dir='data_dir')  # Replace 'YOUR_TELEGRAM_BOT_TOKEN', 'YOUR_CHAT_ID', and 'data_dir' with the actual values
dona_dev.main_loop()


c:\Users\smart\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\smart\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


All necessary libraries are installed.
Cleared previous messages. Last update ID: 265526510
Sent message: Do you want to use package-based data or custom data? (package/custom), Response: {'ok': True, 'result': {'message_id': 734, 'from': {'id': 7166875544, 'is_bot': True, 'first_name': 'anorak', 'username': 'gladiator_anorak_bot'}, 'chat': {'id': 5255840420, 'first_name': 'Anorak', 'type': 'private'}, 'date': 1722706347, 'text': 'Do you want to use package-based data or custom data? (package/custom)'}}
Sent message: Choose a dataset (mnist/cifar10):, Response: {'ok': True, 'result': {'message_id': 736, 'from': {'id': 7166875544, 'is_bot': True, 'first_name': 'anorak', 'username': 'gladiator_anorak_bot'}, 'chat': {'id': 5255840420, 'first_name': 'Anorak', 'type': 'private'}, 'date': 1722706393, 'text': 'Choose a dataset (mnist/cifar10):'}}
Sent message: No logs found., Response: {'ok': True, 'result': {'message_id': 738, 'from': {'id': 7166875544, 'is_bot': True, 'first_name': 'anorak'